In [1]:
!python -m pip install --user --upgrade pip

     |████████████████████████████████| 1.6MB 5.8MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 159 kB 5.7 MB/s 
     |████████████████████████████████| 636 kB 17.1 MB/s 
     |████████████████████████████████| 1.5 MB 18.7 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
  distutils: /usr/lib/python3/dist-packages
  sysconfig: /usr/lib/python3.7/site-packages
  user = False
  home = None
  root = None
  prefix = None
  distutils: /usr/lib/python3/dist-packages
  sysconfig: /usr/lib/python3.7/site-packages
  user = False
  home = None
  root = None
  prefix = None
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 6.3 

In [9]:
import kfp
from kfp import dsl
import kfp.components as comp

In [10]:
def load_data_op():
    return dsl.ContainerOp(
        name = 'Load Data',
        image = 'mavencodevv/load_bird:v.0.1',
        arguments = [],
        file_outputs={
            'img_folder': '/load_data/img_folder'
        }  
    )

In [11]:
def km_op(img_folder):
    return dsl.ContainerOp(
        name = 'Keras Model',
        image = 'mavencodevv/km_bird:v.0.1',
        arguments = ['--img_folder', img_folder],
        file_outputs={
            'km': '/keras_model/km.h5',
            'model_json': '/keras_model/model_json',
            'keras_metrics': '/keras_model/keras_metrics'
        }
    )

In [16]:
def pyt_op(img_folder):
    return dsl.ContainerOp(
        name = 'Pytorch Model',
        image = 'mavencodevv/pyt_bird:v.0.1',
        arguments = ['--img_folder', img_folder],
        file_outputs={
            'pytorch_model': '/pyt_model/pytorch_model.pt',
            'pytorch_metrics': '/pyt_model/pytorch_metrics'
        }  
    )

In [17]:
def eval_op(img_folder, keras_metrics, model_json, km_model, pytorch_metrics, pytorch_model):
    return dsl.ContainerOp(
        name = 'Evaluating Models',
        image = 'mavencodevv/eval_bird:v.0.1',
        arguments = ['--img_folder', img_folder, 
                     '--keras_metrics', keras_metrics,
                     '--model_json', model_json,
                     '--km_model', km_model,
                     '--pytorch_metrics', pytorch_metrics,
                     '--pytorch_model', pytorch_model
                     ]
 
    )

In [18]:
@dsl.pipeline(
    name='Bird Image Classification',
   description='An ML pipeline which trains pytorch and keras models in image classification of birds'
)

# Define parameters to be fed into pipeline
def image_pipeline():

  volume_op = dsl.VolumeOp(
    name="data_volume",
    resource_name="data-volume",
    size="1Gi",
    modes=dsl.VOLUME_MODE_RWO)
  
  _load_data_op = load_data_op().add_pvolumes({'/mnt': volume_op.volume})

  _km_op = km_op(dsl.InputArgumentPath(_load_data_op.outputs['img_folder'])).add_pvolumes({'/mnt': _load_data_op.pvolume})

  _pyt_op = pyt_op(dsl.InputArgumentPath(_load_data_op.outputs['img_folder'])).add_pvolumes({'/mnt': _load_data_op.pvolume})

  _eval_op = eval_op(dsl.InputArgumentPath(_load_data_op.outputs['img_folder']),
                     dsl.InputArgumentPath(_km_op.outputs['keras_metrics']),
                     dsl.InputArgumentPath(_km_op.outputs['model_json']),
                     dsl.InputArgumentPath(_km_op.outputs['km']),
                     dsl.InputArgumentPath(_pyt_op.outputs['pytorch_metrics']),
                     dsl.InputArgumentPath(_pyt_op.outputs['pytorch_model'])).add_pvolumes({'/mnt': _load_data_op.pvolume})


In [19]:
# Compile pipeline to generate compressed YAML definition of the pipeline.
experiment_name = 'birdproject'

kfp.compiler.Compiler().compile(image_pipeline, '{}.yaml'.format(experiment_name))

/root/.local/lib/python3.7/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,
